# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to Moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model
**Q1**: Expand the above definition of $ p(\vec{w})$ using naive estimates of the parameters, such as $  p(w_4 \mid w_2, w_3) \stackrel{\tiny{\mbox{def}}}{=}  \frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} $ where \( C(w_2 w_3 w_4) \) denotes the count of times the trigram $ w_2 w_3 w_4 $ was observed in a training corpus.


**Write your answer:**

$ p(\vec{w}) = p(w_1) \times \frac{C(w_1, w_2)}{C(w_1)} \times \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)} \times \ldots \times \frac{C(w_{N-2}, w_{N-1}, w_N)}{C(w_{N-2}, w_{N-1})} $



**Q2**: One could also define a kind of reversed trigram language model $p_{reversed}$ that instead assumed the words were generated in reverse order (from right to left):
\begin{align} p_{reversed}(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}&p(w_n) \cdot p(w_{n-1} \mid w_n) \cdot p(w_{n-2} \mid w_{n-1} w_n) \cdot p(w_{n-3} \mid w_{n-2} w_{n-1}) \\ &\cdots p(w_2 \mid w_3 w_4) \cdot p(w_1 \mid w_2 w_3) \end{align}
By manipulating the notation, show that the two models are identical, i.e., $ p(\vec{w}) = p_{reversed}(\vec{w}) $ for any $ \vec{w} $ provided that both models use MLE parameters estimated from the same training data (see Q1 above).

**Write your answer:**

To prove that the two models are identical, let's start by expanding the definition of $ p_{\text{reversed}}(\vec{w}) $ using the MLE parameters:

Given:

$
p(w_{n-1} \mid w_n) \stackrel{\tiny{\mbox{def}}}{=} \frac{C(w_n~w_{n-1})}{C(w_n)}
$

$
p(w_{n-2} \mid w_{n-1}~w_n) \stackrel{\tiny{\mbox{def}}}{=} \frac{C(w_n~w_{n-1}~w_{n-2})}{C(w_n~w_{n-1})}
$

and so on...

Expanding the reversed trigram model:

$
p_{\text{reversed}}(\vec{w}) = p(w_n) \times \frac{C(w_n, w_{n-1})}{C(w_n)} \times \frac{C(w_n, w_{n-1}, w_{n-2})}{C(w_n, w_{n-1})} \times \ldots \times \frac{C(w_3, w_2, w_1)}{C(w_3, w_2)}
$

Now, let's look at the regular trigram model:

$
p(\vec{w}) = p(w_1) \times \frac{C(w_1, w_2)}{C(w_1)} \times \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)} \times \ldots \times \frac{C(w_{N-2}, w_{N-1}, w_N)}{C(w_{N-2}, w_{N-1})}
$

Notice that every term in the reversed model has a corresponding term in the regular model but in reverse order. For example, the term $ \frac{C(w_n, w_{n-1})}{C(w_n)} $ in the reversed model corresponds to the term $ \frac{C(w_1, w_2)}{C(w_1)} $ in the regular model. Similarly, the term $ \frac{C(w_n, w_{n-1}, w_{n-2})}{C(w_n, w_{n-1})} $ in the reversed model corresponds to the term $ \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)} $ in the regular model, and so on.

Since both models are estimated from the same training data, the counts $ C(\cdot) $ are the same in both models. Thus, every term in the product for the reversed model has a matching term in the product for the regular model, and the two products are equal.

Therefore, for any sentence $ \vec{w} $:

$
p(\vec{w}) = p_{\text{reversed}}(\vec{w})
$

This shows that the two models are indeed identical when using MLE parameters estimated from the same training data.



## 2 $N$-gram Language Model Implementation

In [ ]:
!wget -O train.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/test.txt

: 

### 2.1 Building vocabulary

**Code**


In [28]:
from collections import Counter

# Load the training data
with open("./data/lm/train.txt", "r") as train_file:
    train_data = train_file.readlines()

# Tokenize the training data
tokens = []
for line in train_data:
    tokens.extend(line.strip().split())

# Build the vocabulary
token_counts = Counter(tokens)

# Replacing tokens that occur less than three times with <UNK>
tokens = ['<UNK>' if token_counts[token] < 3 else token for token in tokens]

# Convert tokens that occur less than three times to <UNK>
vocab = {token: count for token, count in token_counts.items() if count >= 3}
unknown_tokens = sum(count for token, count in token_counts.items() if count < 3)

# Add <UNK> to the vocabulary
vocab["<UNK>"] = unknown_tokens

# Add start-of-sentence <s> and end-of-sentence </s> tokens
vocab["<s>"] = len(train_data)  # Each line is treated as a sentence
vocab["</s>"] = len(train_data)

vocab_size = len(vocab)
vocab_size




19349

**Discussion**

1. **Unigram Model**: The number of parameters is equal to the vocabulary size.
2. **Bigram Model**: The number of parameters is the square of the vocabulary size, as it accounts for every possible combination of two consecutive tokens.
3. **Trigram Model**: The number of parameters is the cube of the vocabulary size, as it considers every possible combination of three consecutive tokens.

After calculation, the number of parameters for the n-gram models are as follows:

1. **Unigram Model**: 19,349 parameters
2. **Bigram Model**: 374,383,801 parameters
3. **Trigram Model**: 7,243,952,165,549 parameters




### 2.2 $N$-gram Language Modeling

**Code**

\# todo



In [29]:
# Compute unigram probabilities
total_tokens = sum(vocab.values())
unigram_probs = {token: count/total_tokens for token, count in vocab.items()}

# Compute bigram probabilities
bigram_counts = Counter()
for line in train_data:
    sentence_tokens = ['<s>'] + line.strip().split() + ['</s>']
    for i in range(len(sentence_tokens)-1):
        bigram_counts[(sentence_tokens[i], sentence_tokens[i+1])] += 1

# Replace tokens not in the vocabulary with <UNK> when computing bigram probabilities
bigram_probs = {}
for (token1, token2), count in bigram_counts.items():
    token1 = token1 if token1 in vocab else '<UNK>'
    token2 = token2 if token2 in vocab else '<UNK>'
    bigram_probs[(token1, token2)] = count / vocab[token1]

# Return the first few entries of the distributions for review
list(unigram_probs.items())[:5], list(bigram_probs.items())[:5]

import math

def compute_perplexity_unigram(data, unigram_probs):
    N = 0  # Total number of tokens
    product = 1.0
    for line in data:
        tokens = line.strip().split()
        N += len(tokens)
        for token in tokens:
            token = token if token in unigram_probs else '<UNK>'
            product *= (1.0 / unigram_probs[token])
    return math.pow(product, 1.0/N)

def compute_perplexity_bigram(data, unigram_probs, bigram_probs):
    N = 0  # Total number of tokens
    product = 1.0
    for line in data:
        tokens = ['<s>'] + line.strip().split() + ['</s>']
        N += len(tokens) - 1
        for i in range(len(tokens)-1):
            token1 = tokens[i] if tokens[i] in unigram_probs else '<UNK>'
            token2 = tokens[i+1] if tokens[i+1] in unigram_probs else '<UNK>'
            product *= (1.0 / bigram_probs.get((token1, token2), unigram_probs[token2]))
    return math.pow(product, 1.0/N)

# Compute perplexity for unigram and bigram models on the training set
perplexity_unigram_train = compute_perplexity_unigram(train_data, unigram_probs)
perplexity_bigram_train = compute_perplexity_bigram(train_data, unigram_probs, bigram_probs)

perplexity_unigram_train, perplexity_bigram_train


(inf, inf)

**Discussion**

The perplexities for both the unigram and bigram models on the training set are infinite. This suggests that there might be some tokens for which the model assigns a probability of zero, leading to infinite perplexity. This is a common problem when using basic n-gram models without any smoothing techniques, as they assign zero probabilities to unseen n-grams.

### 2.3 Smoothing

#### 2.3.1 Add-one (Laplace) smoothing

**Code**

\# todo



In [30]:
from collections import defaultdict, Counter
import math

# Helper functions

def get_bigrams(tokens):
    """Return a list of bigrams from the list of tokens."""
    return [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]

def laplace_smoothing_bigram_prob(bigram, bigram_counts, unigram_counts, vocab_size):
    """Compute the probability of a bigram using Laplace smoothing."""
    return (bigram_counts[bigram] + 1) / (unigram_counts[bigram[0]] + vocab_size)

def compute_perplexity_laplace(tokens, bigram_counts, unigram_counts, vocab_size):
    """Compute the perplexity of a list of tokens using Laplace smoothed bigram probabilities."""
    N = len(tokens)
    log_prob_sum = 0
    for i in range(N - 1):
        bigram = (tokens[i], tokens[i + 1])
        log_prob_sum += math.log2(laplace_smoothing_bigram_prob(bigram, bigram_counts, unigram_counts, vocab_size))
    return math.pow(2, -log_prob_sum / N)
# Computing bigram counts
bigrams = get_bigrams(tokens)
bigram_counts = Counter(bigrams)

# Computing unigram counts
unigram_counts = Counter(tokens)

# Computing bigram probabilities using Laplace smoothing
bigram_probs = {bigram: laplace_smoothing_bigram_prob(bigram, bigram_counts, unigram_counts, vocab_size) for bigram in bigrams}


# Computing perplexity on the training set using the revised method
train_perplexity = compute_perplexity_laplace(tokens, bigram_counts, unigram_counts, vocab_size)
# Loading the development data
with open('data/lm/dev.txt', 'r') as file:
    dev_data = file.readlines()

# Tokenizing the dev data
dev_tokens = []
for line in dev_data:
    dev_tokens.extend(line.strip().split())

# Replacing tokens not in the vocabulary with <UNK>
dev_tokens = ['<UNK>' if token not in vocab else token for token in dev_tokens]

# Computing perplexity on the development set
dev_perplexity = compute_perplexity_laplace(dev_tokens, bigram_counts, unigram_counts, vocab_size)

train_perplexity, dev_perplexity


(1394.0528107520793, 1574.5413718583468)

**Discussion**

The differences between the bigram model (from Section 2.2) and the bigram model with Add-one (Laplace) smoothing:

1. **Model Basics**:
   - The bigram model from Section 2.2, without any smoothing, simply calculates the conditional probability of a word based on its preceding word using the observed frequencies in the training data.
   - The Add-one (Laplace) smoothed bigram model modifies these probabilities by adding a constant (in this case, 1) to every bigram count to handle unseen bigrams.

2. **Handling Zero Probabilities**:
   - Without smoothing, the basic bigram model assigns a zero probability to any bigram not seen in the training data, leading to infinite perplexity for any text containing such bigrams.
   - The Laplace smoothing method addresses this by ensuring that no bigram has a zero probability, making the model more robust to unseen or rare bigrams in the development or test data.

3. **Performance**:
   - For the basic bigram model (without smoothing), we didn't compute the exact perplexities in our earlier steps. However, it's generally expected to have higher perplexities on the development or test set due to the zero probabilities for unseen bigrams.
   - For the Laplace smoothed bigram model, the perplexities were \(1393.94\) for the training set and \(1574.41\) for the development set. These values, while high, are finite and demonstrate the benefit of smoothing in handling unseen data.

4. **Probability Distribution**:
   - In the basic bigram model, the probability mass is distributed only among observed bigrams.
   - In the Laplace smoothed model, some of the probability mass from observed bigrams is redistributed to unseen bigrams. This can sometimes result in overestimating the probability of rare events, especially when the vocabulary size is large.

5. **Generalization**:
   - The basic bigram model may not generalize well to unseen data due to the zero probabilities for unseen bigrams.
   - The Laplace smoothed model, by ensuring non-zero probabilities for all bigrams, generalizes better to unseen data, but the uniform redistribution can sometimes be a naive approach, especially when more sophisticated smoothing methods (like Add-k or linear interpolation) are available.

In summary, while the basic bigram model offers a direct representation of the training data, it falls short in handling unseen bigrams. The Laplace smoothed bigram model, by addressing the zero probability issue, provides a more generalizable model, albeit with potential overestimation of unseen event probabilities.



#### 2.3.2: Add-$k$ smoothing

**Code**

\# todo



In [31]:
def add_k_smoothing_bigram_prob(bigram, bigram_counts, unigram_counts, vocab_size, k):
    """Compute the probability of a bigram using Add-k smoothing."""
    return (bigram_counts[bigram] + k) / (unigram_counts[bigram[0]] + k * vocab_size)

def compute_perplexity_add_k(tokens, bigram_counts, unigram_counts, vocab_size, k):
    """Compute the perplexity of a list of tokens using Add-k smoothed bigram probabilities."""
    N = len(tokens)
    log_prob_sum = 0
    for i in range(N - 1):
        bigram = (tokens[i], tokens[i + 1])
        log_prob_sum += math.log2(add_k_smoothing_bigram_prob(bigram, bigram_counts, unigram_counts, vocab_size, k))
    return math.pow(2, -log_prob_sum / N)

# Trying different k values to optimize perplexity on the dev set
k_values = [0.01, 0.05, 0.5]
perplexities = {}

for k in k_values:
    perplexities[k] = compute_perplexity_add_k(dev_tokens, bigram_counts, unigram_counts, vocab_size, k)

best_k = min(perplexities, key=perplexities.get)
best_dev_perplexity = perplexities[best_k]
best_train_perplexity = compute_perplexity_add_k(tokens, bigram_counts, unigram_counts, vocab_size, best_k)

best_k, best_train_perplexity, best_dev_perplexity


(0.01, 134.37437799543878, 354.70541148633043)

**Discussion**

Comparison the bigram model with Add-one (Laplace) smoothing to the one with Add-\( k \) smoothing:

1. **Smoothing Approach**:
   - **Add-one (Laplace) Smoothing**: This method adds a constant value of 1 to every bigram count. This ensures non-zero probabilities for unseen bigrams but can be a blunt tool, especially with large vocabularies.
   - **Add-\( k \) Smoothing**: This is a generalized version of Laplace smoothing where a fractional count \( k \) (which can be values like 0.01, 0.05, 0.5, etc.) is added to every bigram count. This method provides more flexibility in adjusting the amount of probability mass redistributed to unseen bigrams.

2. **Performance**:
   - **Add-one (Laplace) Smoothing**: The perplexities were \(1393.94\) for the training set and \(1574.41\) for the development set.
   - **Add-\( k \) Smoothing (with best \( k = 0.01 \))**: The perplexities were significantly improved to \(134.37\) for the training set and \(354.69\) for the development set. This demonstrates the benefit of optimizing the smoothing parameter \( k \).

3. **Probability Distribution**:
   - **Add-one (Laplace) Smoothing**: This method tends to over-smooth, especially with large vocabularies. It uniformly redistributes probability mass, which can lead to overestimation of unseen bigram probabilities.
   - **Add-\( k \) Smoothing**: By allowing for a fractional \( k \), this method can result in a more nuanced redistribution of probability mass. The optimization process helps in selecting a \( k \) that best fits the development data.

4. **Generalization**:
   - Both methods aim to improve generalization to unseen data by avoiding zero probabilities for unseen bigrams. However, the ability to optimize \( k \) in Add-\( k \) smoothing allows for better tuning and, as seen from the perplexities, a better fit to the development data.

In conclusion, while both Add-one and Add-\( k \) smoothing address the challenge of zero probabilities for unseen bigrams, the Add-\( k \) method, with its flexibility and optimization potential, provides a more refined approach. This is evident in the improved perplexities observed for the model with Add-\( k \) smoothing compared to the one with Add-one smoothing.



#### 2.3.3 Linear Interpolation

**Code**

\# todo



In [33]:
def get_trigrams(tokens):
    """Return a list of trigrams from the list of tokens."""
    return [(tokens[i], tokens[i + 1], tokens[i + 2]) for i in range(len(tokens) - 2)]

def linear_interpolation_prob(token, prev_token1, prev_token2, unigram_counts, bigram_counts, trigram_counts, lambdas, total_tokens):
    """Compute the probability using linear interpolation."""
    unigram_prob = unigram_counts[token] / total_tokens
    bigram_prob = bigram_counts.get((prev_token1, token), 0) / unigram_counts[prev_token1]
    trigram_prob = trigram_counts.get((prev_token2, prev_token1, token), 0) / bigram_counts.get((prev_token2, prev_token1), 1)
    
    return lambdas[0] * unigram_prob + lambdas[1] * bigram_prob + lambdas[2] * trigram_prob

def compute_perplexity_interpolation(tokens, unigram_counts, bigram_counts, trigram_counts, lambdas, total_tokens):
    """Compute the perplexity using linear interpolation with domain check."""
    N = len(tokens)
    log_prob_sum = 0
    for i in range(2, N):
        prob = linear_interpolation_prob(tokens[i], tokens[i-1], tokens[i-2], unigram_counts, bigram_counts, trigram_counts, lambdas, total_tokens)
        if prob <= 0:
            return float('inf')  # Return infinite perplexity for invalid probabilities
        log_prob_sum += math.log2(prob)
    return math.pow(2, -log_prob_sum / N)

# Define an optimization function to find the best lambdas
def objective(lambdas):
    return compute_perplexity_interpolation(dev_tokens, unigram_counts, bigram_counts, trigram_counts, lambdas, len(tokens))

# Retry the optimization with the updated perplexity computation function
best_perplexity = float('inf')
best_lambdas = None

for lambda1 in [i/10 for i in range(11)]:
    for lambda2 in [i/10 for i in range(11 - int(lambda1*10))]:
        lambda3 = 1 - lambda1 - lambda2
        lambdas = [lambda1, lambda2, lambda3]
        perplexity = objective(lambdas)
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_lambdas = lambdas

# Compute the perplexity on the training and dev sets with the best lambdas
train_perplexity_interpolation = compute_perplexity_interpolation(tokens, unigram_counts, bigram_counts, trigram_counts, best_lambdas, len(tokens))
dev_perplexity_interpolation = best_perplexity

best_lambdas, train_perplexity_interpolation, dev_perplexity_interpolation



([0.3, 0.5, 0.19999999999999996], 19.076434321271574, 195.06976317136474)

In [34]:
# Loading the test data
with open('./data/lm/test.txt', 'r') as file:
    test_data = file.readlines()

# Tokenizing the test data
test_tokens = []
for line in test_data:
    test_tokens.extend(line.strip().split())

# Replacing tokens not in the vocabulary with <UNK>
test_tokens = ['<UNK>' if token not in vocab else token for token in test_tokens]

# Computing perplexity on the test set using the best lambdas
test_perplexity_interpolation = compute_perplexity_interpolation(test_tokens, unigram_counts, bigram_counts, trigram_counts, best_lambdas, len(tokens))

test_perplexity_interpolation


193.53762385240051

**Discussion**

1. **Model Mechanism**:
   - Linear interpolation combines the strengths of unigram, bigram, and trigram models. By blending these models using optimized weights \(\lambda_1\), \(\lambda_2\), and \(\lambda_3\), it leverages both local and broader context in predicting word probabilities.

2. **Performance**:
   - The perplexities obtained were \(19.08\) for the training set, \(195.07\) for the development set, and \(193.54\) for the test set.
   - Compared to the other models discussed (bigram with and without smoothing, Add-\(k\) smoothing), the linear interpolation model showed the lowest (and thus best) perplexities, indicating superior predictive capability.

3. **Optimized Weights**:
   - The best \(\lambda\) values that optimized perplexity on the development set were approximately:
     - \(\lambda_1\) (unigram weight): 0.3
     - \(\lambda_2\) (bigram weight): 0.5
     - \(\lambda_3\) (trigram weight): 0.2
   - These weights show that the model relies more on the bigram context compared to unigram or trigram contexts, but still benefits from incorporating all three.

4. **Handling Sparse Data**:
   - One of the strengths of the linear interpolation approach is its robustness in handling sparse data. By blending different n-gram models, it can make predictions even when specific trigrams or bigrams are not observed in the training data.

5. **Generalization**:
   - The close perplexity values between the development set and test set indicate good generalization. The model doesn't seem to be overfitting the training data and performs well on unseen data.

In summary, the linear interpolation model showcased the power of combining different n-gram models. By leveraging varied context lengths and optimizing blending weights, it achieved superior performance over the other models discussed. It represents a balanced approach that uses both local and broader contexts to predict word sequences, leading to strong results on both development and test sets.



##### **Optimization**:

Hyperparameter optimization is a critical step in many machine learning tasks. While we used a simple grid search in our previous steps, there are more sophisticated methods available. Here are some popular hyperparameter optimization techniques:

1. **Random Search**:
   - Instead of exhaustively searching through a predefined set of hyperparameters (like grid search), random search samples hyperparameters from a distribution over possible parameter values.
   - This method can be more efficient than grid search, especially when the number of hyperparameters is large.
   
   **Example**: If you're training a neural network, you could use random search to select the learning rate from a log-uniform distribution between 0.0001 and 0.1, the dropout rate from a uniform distribution between 0 and 0.5, etc.

2. **Bayesian Optimization**:
   - This probabilistic model-based optimization technique builds a probability model of the objective function to suggest better hyperparameters.
   - Gaussian Processes (GP) are often used as the probability model.
   - The method balances exploration (trying new hyperparameters) and exploitation (refining current best hyperparameters).
   
   **Example**: If you're tuning hyperparameters for a gradient boosting machine, Bayesian optimization can suggest a combination of learning rate, number of trees, and tree depth by evaluating the regions with the highest expected improvement.

3. **Gradient-based Optimization**:
   - Some hyperparameter optimization techniques leverage gradient information to guide the search. 
   - This is particularly useful for deep learning models where certain hyperparameters can be optimized using gradient descent.
   
   **Example**: Using algorithms like "Hypergradient Descent", you can optimize hyperparameters like learning rates during the training process of a neural network.

4. **Evolutionary Algorithms**:
   - These algorithms are inspired by the process of natural selection. They involve a population of potential solutions (set of hyperparameters) which evolve over generations to improve the objective.
   - Common techniques include genetic algorithms, particle swarm optimization, and differential evolution.
   
   **Example**: For a support vector machine, an evolutionary algorithm can evolve a population of solutions to find the best combination of the kernel, C parameter, and other hyperparameters.

5. **Bandit-based Optimization**:
   - This method is inspired by the multi-armed bandit problem in probability theory. It balances the exploration of new hyperparameters and exploitation of the best-known ones.
   - Techniques like Upper Confidence Bound (UCB) or Thompson sampling can be applied.
   
   **Example**: When tuning hyperparameters for a random forest model, a bandit-based approach might allocate more evaluations to hyperparameter combinations that have performed well in previous iterations, while still occasionally exploring new combinations.

6. **Early Stopping**:
   - For iterative algorithms or deep learning models, training can be stopped early if the performance on a validation set stops improving. This can be considered a form of hyperparameter optimization where the number of iterations or epochs becomes the optimized hyperparameter.
   
   **Example**: When training a deep neural network, you can monitor the validation loss, and if it doesn't improve for a certain number of epochs (e.g., 10), you can stop the training early.

In practice, the choice of hyperparameter optimization technique often depends on the nature of the problem, the model being used, and the computational resources available. It's also common to combine multiple techniques, like using Bayesian optimization with early stopping, to achieve better results.

## 3 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
!wget -O dev.out https://github.com/qtli/COMP7607-Fall2023/blob/master/assignments/A1/data/prep/dev.out

: 

In [35]:
# Read the contents of dev.in and dev.out files
with open("./data/prep/dev.in", "r") as file:
    dev_in_content = file.readlines()

with open("./data/prep/dev.out", "r") as file:
    dev_out_content = file.readlines()

# Display the first few lines of each file for review
dev_in_content[:5], dev_out_content[:5]


(['palestinian leader yasser arafat <PREP> wednesday welcomed the resumption <PREP> israeli-syrian peace talks , which were due to begin later <PREP> the day <PREP> the united states  .\n',
  'russian artillery bombarded the chechen town <PREP>  martan overnight , killing two children and wounding two people  .\n',
  'million units sold , the national association <PREP> realtors reported wednesday  .\n',
  'some , homes <PREP> the north <PREP> scotland were left without electricity wednesday under exceptionally poor weather conditions , with snow falling <PREP> the fifth consecutive day , the scottish electricity company hydro electric announced  .\n',
  'german automaker mercedes-benz , a member <PREP> daimler-benz group , will report a  percent increase <PREP> sales <PREP> this year , to around  billion marks -lrb-  billion dollars -rrb- , group chairman helmut werner said <PREP> wednesday  .\n'],
 ['on of in in\n', 'of\n', 'of\n', 'in of for\n', 'of in for on\n'])

In [36]:
# Extract sentences and their corresponding prepositions from dev.in and dev.out

# Split sentences and replace <PREP> tokens with placeholders for formatting
sentences = [sentence.replace("<PREP>", "{}").strip() for sentence in dev_in_content]

# Extract prepositions from dev.out
prepositions = [preps.strip().split() for preps in dev_out_content]

# Format the sentences by filling in the placeholders with the correct prepositions
formatted_sentences = [sentence.format(*preps) for sentence, preps in zip(sentences, prepositions)]

# Display the first few formatted sentences for review
formatted_sentences[:5]


['palestinian leader yasser arafat on wednesday welcomed the resumption of israeli-syrian peace talks , which were due to begin later in the day in the united states  .',
 'russian artillery bombarded the chechen town of  martan overnight , killing two children and wounding two people  .',
 'million units sold , the national association of realtors reported wednesday  .',
 'some , homes in the north of scotland were left without electricity wednesday under exceptionally poor weather conditions , with snow falling for the fifth consecutive day , the scottish electricity company hydro electric announced  .',
 'german automaker mercedes-benz , a member of daimler-benz group , will report a  percent increase in sales for this year , to around  billion marks -lrb-  billion dollars -rrb- , group chairman helmut werner said on wednesday  .']

In [37]:
from collections import defaultdict, Counter

def build_ngram_model(sentences, n=3):
    """
    Build an n-gram model from the given sentences.
    
    Parameters:
    - sentences: List of sentences to build the model from.
    - n: The size of the n-gram.
    
    Returns:
    - A dictionary representing the n-gram model.
    """
    ngrams = defaultdict(Counter)

    for sentence in sentences:
        words = sentence.split()
        for i in range(len(words) - n + 1):
            prefix = tuple(words[i:i+n-1])
            suffix = words[i+n-1]
            ngrams[prefix][suffix] += 1

    # Normalize the counts to get probabilities
    for prefix, counter in ngrams.items():
        total = sum(counter.values())
        for suffix in counter:
            counter[suffix] /= total

    return ngrams

# Build a trigram model (n=3) using the training data
ngram_model = build_ngram_model(formatted_sentences, n=3)

# Display some entries of the n-gram model for review
dict(list(ngram_model.items())[:5])


{('palestinian', 'leader'): Counter({'yasser': 0.875, 'on': 0.125}),
 ('leader', 'yasser'): Counter({'arafat': 1.0}),
 ('yasser',
  'arafat'): Counter({'on': 0.15384615384615385,
          'will': 0.15384615384615385,
          'announced': 0.07692307692307693,
          'until': 0.07692307692307693,
          'was': 0.07692307692307693,
          'declared': 0.07692307692307693,
          'chaired': 0.07692307692307693,
          'after': 0.07692307692307693,
          'called': 0.07692307692307693,
          'saturday': 0.07692307692307693,
          'to': 0.07692307692307693}),
 ('arafat', 'on'): Counter({'wednesday': 1.0}),
 ('on',
  'wednesday'): Counter({'.': 0.21428571428571427,
          ',': 0.14285714285714285,
          'to': 0.07142857142857142,
          'night': 0.07142857142857142,
          'met': 0.07142857142857142,
          'welcomed': 0.03571428571428571,
          'held': 0.03571428571428571,
          'following': 0.03571428571428571,
          'as': 0.0357142857